In [1]:
# ==============================================================================
# HÜCRE 1: KÜTÜPHANE KURULUMU
# Amaç: Proje için gerekli olan tüm Python kütüphanelerini kurmak ve
#       Google Colab ortamının bu kütüphaneleri doğru şekilde tanımasını sağlamak.
# ==============================================================================
print("▶️ HÜCRE 1: Gerekli kütüphaneler kuruluyor...")

# !pip komutu, Colab'in sanal makinesine paket kurmamızı sağlar.
# -U (upgrade) bayrağı, paketlerin en son sürümüne güncellenmesini sağlar.
# -q (quiet) bayrağı, kurulum sırasında oluşan uzun çıktı metinlerini gizler.
# Kütüphaneleri daha okunaklı olması için ayırarak kuruyoruz:
!pip install -U -q \
    langchain \
    langchain-google-genai \
    langchain-huggingface \
    langchain-chroma \
    sentence-transformers \
    pandas \
    tqdm

# KURULAN KÜTÜPHANELERİN GÖREVLERİ:
# ------------------------------------
# langchain: RAG (Retrieval-Augmented Generation) mimarisinin "beyni".
#            Veri akışını (chain), prompt'ları ve modelleri yöneten ana kütüphane.
#
# langchain-google-genai: LangChain'in Google Gemini (gemini-pro) modeliyle
#                         iletişim kurmasını sağlayan entegrasyon paketi. LLM'imiz budur.
#
# langchain-huggingface: LangChain'in Hugging Face modellerini kullanmasını sağlar.
#                        Bunu, şarkı sözlerini vektöre çevirecek olan Embedding
#                        modeli ('all-MiniLM-L6-v2') için kullanıyoruz.
#
# langchain-chroma: LangChain'in ChromaDB vektör veritabanı ile konuşmasını sağlar.
#                   (Veritabanına vektör ekleme, sorgulama vb. için)
#
# sentence-transformers: Şarkı sözlerini (metin) anlamsal vektörlere (sayısal temsillere)
#                        dönüştürmek için kullandığımız 'all-MiniLM-L6-v2'
#                        modelini içeren asıl kütüphane.
#
# pandas: Veri setini (şarkı sözleri, sanatçılar) okumak, işlemek ve
#         zenginleştirmek için kullanılan temel veri analizi kütüphanesi.
#
# tqdm: Özellikle binlerce şarkı sözünü işlerken ilerleme durumunu
#       gösteren bir ilerleme çubuğu (progress bar) oluşturur.

print("\n✅ Kurulum tamamlandı.")

# --- ÇOK ÖNEMLİ NOT ---
# Google Colab, oturum başladığında belirli kütüphaneleri hafızaya yükler.
# 'pip install -U' ile bu kütüphanelerin yeni sürümlerini kursak bile,
# Colab 'Restart session' yapılana kadar hafızadaki eski sürümleri
# kullanmaya devam edebilir.
#
# Bu durum, özellikle 'langchain' gibi hızla güncellenen kütüphanelerde
# 'ImportError' veya 'AttributeError' gibi hatalara yol açar.
# Oturumu yeniden başlatmak (Restart session), Colab'in yeni kurulan
# paketleri tanımasını ve doğru sürümü yüklemesini garanti altına alır.
print("\n⚠️ >>>>> KESİNLİKLE YAP: Şimdi menüden 'Runtime' -> 'Restart session' diyerek oturumu yeniden başlat! <<<<< ⚠️")

▶️ HÜCRE 1: Gerekli kütüphaneler kuruluyor...

✅ Kurulum tamamlandı.

⚠️ >>>>> KESİNLİKLE YAP: Şimdi menüden 'Runtime' -> 'Restart session' diyerek oturumu yeniden başlat! <<<<< ⚠️


In [1]:
# ==============================================================================
# HÜCRE 2: GEREKLİ MODÜLLERİN İÇE AKTARILMASI VE API ANAHTARI
# Amaç: (Hücre 1'de kurduğumuz) kütüphanelerden ihtiyaç duyduğumuz
#       spesifik fonksiyonları/sınıfları kodumuza dahil etmek ve
#       Google API anahtarını güvenli bir şekilde Colab ortamına tanıtmak.
# ==============================================================================
print("▶️ HÜCRE 2: Gerekli modüller içe aktarılıyor ve API anahtarı ayarlanıyor...")

# --- Standart Kütüphaneler ---
import os
    # 'os' (Operating System): İşletim sistemiyle ilgili işlemler yapmak için kullanılır.
    # Örneğin, klasörlerin var olup olmadığını (os.path.exists) kontrol etmek veya
    # 'environment variables' (ortam değişkenleri) içine API anahtarımızı eklemek için.

import getpass
    # 'getpass': Kullanıcıdan parola/API anahtarı gibi gizli bilgileri
    # terminalde/notebook hücresinde GÖRÜNMEDEN (yankılanmadan) almayı sağlar.
    # Bu, API anahtarımızın kopyalanmasını veya ekran görüntülerinde görünmesini engeller.

import pandas as pd
    # 'pandas': Veri setimizi (.csv, .txt) okumak, DataFrame adı verilen
    # tablosal bir yapıda tutmak ve manipüle etmek (örn: 'sanatçı' sütunu eklemek) için.

from tqdm import tqdm
    # 'tqdm' (tahammül): Uzun süren döngüler (örn: binlerce şarkıyı okuma) için
    # bir ilerleme çubuğu (progress bar) gösterir. Ne kadar iş kaldığını görmemizi sağlar.

from google.colab import files
    # 'files': Google Colab'e özgü bu modül, 'kaggle.json' gibi dosyaları
    # lokal bilgisayarımızdan Colab ortamına yüklememizi sağlar (files.upload()).

# --- LangChain Kütüphaneleri (RAG Mimarisi için) ---

# DÜZELTME: Doğru ve en güncel import yolu eklendi
from langchain_text_splitters import RecursiveCharacterTextSplitter
    # RAG için kritik bir araç. Uzun şarkı sözlerini, modelin anlayabileceği
    # daha küçük ve anlamlı parçalara (chunks) bölmek için kullanılır.
    # 'Recursive' olması, metni "anlamlı" yerlerden (örn: paragraf, satır)
    # bölmeye çalışması anlamına gelir.

# DÜZELTME: Document'ın da doğru import yolu
from langchain_core.documents import Document
    # LangChain'in standart veri yapısı. Şarkı sözlerini (page_content) ve
    # o söze ait üst veriyi (metadata: sanatçı, şarkı adı) bir arada tutan
    # bir "belge" nesnesi oluşturmamızı sağlar.

from langchain_chroma import Chroma
    # Vektör veritabanımız olan ChromaDB ile etkileşim kurmamızı sağlayan sınıf.
    # Vektörleri kaydetmek (from_documents) ve veritabanını yüklemek (persist_directory)
    # için kullanılır.

from langchain_huggingface import HuggingFaceEmbeddings
    # Şarkı sözü parçalarını (metin) vektörlere (sayılar) dönüştürecek olan
    # Hugging Face modelini (all-MiniLM-L6-v2) yüklemek için kullanılır.

from langchain_google_genai import ChatGoogleGenerativeAI
    # RAG zincirinin son aşamasında, soruyu ve bulunan bağlamı (şarkı sözü)
    # anlayıp cevap verecek olan asıl 'Large Language Model' (LLM) olan
    # Gemini-Pro modelini yüklememizi sağlar.

# RetrievalQA import'u kaldırıldı, çünkü Hücre 4'te LCEL kullanıyoruz.
    # Not: Eski LangChain'de 'RetrievalQA' sınıfı kullanılırdı.
    # Biz ise daha modern, esnek ve güçlü olan LCEL (LangChain Expression Language)
    # boru hattı ('|' operatörü ile) yöntemini (Hücre 4'te) kullanıyoruz.

# --- API Anahtarı Ayarlaması ---

# 'os.environ', Colab ortamının "ortam değişkenlerini" tutan bir sözlüktür.
# Kütüphaneler (örn: langchain_google_genai) API anahtarını otomatik olarak
# "GOOGLE_API_KEY" adlı değişkenden arar.
if "GOOGLE_API_KEY" not in os.environ:
    # Eğer anahtar daha önce ayarlanmamışsa, kullanıcıdan güvenli bir şekilde iste:
    print("API Anahtarı bulunamadı. Lütfen şimdi girin:")

    # 'getpass.getpass' ile kullanıcıdan anahtarı görünmez bir şekilde al
    # ve 'os.environ' içine ata.
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Lütfen Google API Anahtarınızı yapıştırın:")
    print("API Anahtarı başarıyla ayarlandı.")
else:
    # Eğer anahtar zaten ayarlanmışsa (örn: notebook'u tekrar çalıştırıyorsak)
    # tekrar sormaya gerek yok.
    print("API Anahtarı zaten ayarlanmış.")

print("\n✅ Modüller ve API anahtarı hazır.")

▶️ HÜCRE 2: Gerekli modüller içe aktarılıyor ve API anahtarı ayarlanıyor...
API Anahtarı bulunamadı. Lütfen şimdi girin:
Lütfen Google API Anahtarınızı yapıştırın:··········
API Anahtarı başarıyla ayarlandı.

✅ Modüller ve API anahtarı hazır.


In [2]:
# ==============================================================================
# HÜCRE 3: VERİ HAZIRLAMA VE VEKTÖR VERİTABANI OLUŞTURMA (SADECE 1 KEZ ÇALIŞTIR)
# Amaç: Ham veriyi (şarkı sözü .txt dosyaları) Kaggle'dan indirmek,
#       bu veriyi yapılandırmak (Pandas DataFrame),
#       eksik bilgileri (sanatçı adı) manuel bir harita ile zenginleştirmek,
#       veriyi RAG için parçalamak (chunking) ve
#       bu parçaları vektörleştirerek kalıcı bir 'chroma_db' veritabanına kaydetmek.
#
# NOT: Bu hücre, veritabanını SIFIRDAN oluşturur ve 'chroma_db' klasörünün
#      üzerine yazar. Bu nedenle SADECE BİR KEZ çalıştırılmalıdır.
# ==============================================================================
print("▶️ HÜCRE 3: Veri seti hazırlanıyor ve vektör veritabanı SIFIRDAN oluşturuluyor...")

# --- 1. Veri İndirme (Kaggle) ---
# 'Lyrics' klasörü Colab ortamında zaten yoksa, indirme işlemini başlat.
# Bu, hücreyi tekrar çalıştırdığımızda veriyi tekrar indirmemizi engeller.
if not os.path.exists('Lyrics'):
    print("Veri seti 'Lyrics' klasörü bulunamadı, Kaggle'dan indiriliyor...")

    # Kaggle API'sinin çalışması için 'kaggle.json' API anahtar dosyası gerekir.
    # Eğer bu dosya da ortamda yoksa, kullanıcıdan yüklemesini iste.
    if not os.path.exists('kaggle.json'):
        print("Lütfen 'kaggle.json' dosyasını seçin:")
        files.upload() # Colab'in dosya yükleme arayüzünü açar.

    # Kaggle API'sinin 'kaggle.json' dosyasını bulabileceği standart yolu oluştur.
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/

    # 'kaggle.json' dosyasına sadece sahibinin (bizim) okuma/yazma izni ver.
    # Bu, Kaggle API'sinin güvenlik uyarısı vermemesi için gereklidir.
    !chmod 600 ~/.kaggle/kaggle.json

    # Kaggle API'sini kullanarak veri setini indir.
    !kaggle datasets download -d anil1055/turkish-music-spectograms

    # İndirilen .zip dosyasını aç (-q: sessiz, -o: üzerine yaz)
    # ve mevcut dizine çıkart (-d .).
    !unzip -q -o turkish-music-spectograms.zip -d .
    print("Veri seti başarıyla indirildi ve 'Lyrics' klasörüne açıldı.")
else:
    print("Veri seti 'Lyrics' klasörü zaten mevcut, indirme adımı atlandı.")


# --- 2. Veriyi Okuma ve Yapılandırma ---
# Veri seti, 'Lyrics' klasörü altında 'Pop', 'Rock', 'Arabesk' gibi
# alt klasörlerde .txt dosyaları olarak durmaktadır.

song_data = [] # Okunan şarkı bilgilerini depolamak için boş bir liste.
lyrics_folder_path = 'Lyrics' # Ana veri klasörünün yolu.

# 'os.walk', bir klasör ağacında (tüm alt klasörler dahil) gezinmemizi sağlar.
# 'tqdm' ile bu gezinme işlemini bir ilerleme çubuğu ile gösteriyoruz.
for root, dirs, files_list in tqdm(os.walk(lyrics_folder_path), desc="Şarkılar okunuyor"):
    for file in files_list:
        # Sadece .txt uzantılı dosyaları (şarkı sözlerini) işle
        if file.endswith('.txt'):
            # Dosya yolunu (root) parçalayarak ('/') kategori adını al.
            # Örn: 'Lyrics/Pop' -> ['Lyrics', 'Pop'] -> 'Pop'
            path_parts = root.split(os.sep)
            kategori = path_parts[-1] if len(path_parts) > 1 else "Bilinmiyor"

            # Dosya adından uzantıyı (.txt) ayırarak şarkı adını al.
            # Örn: 'Aşkın Mapushane.txt' -> 'Aşkın Mapushane'
            sarki_adi = os.path.splitext(file)[0]

            # Dosyanın tam yolunu oluştur
            file_path = os.path.join(root, file)

            try:
                # Dosyayı 'utf-8' encoding ile aç ve içeriğini (lyrics) oku.
                with open(file_path, 'r', encoding='utf-8') as f:
                    lyrics = f.read()

                    # Şarkı için bir sözlük oluştur
                    song_info = {'kategori': kategori, 'sarki_adi': sarki_adi, 'sozler': lyrics}

                    # Bu sözlüğü ana listemize ekle
                    song_data.append(song_info)
            except Exception:
                # Bazı .txt dosyaları bozuk olabilir veya encoding hatası verebilir.
                # Bu dosyaları atla (pass) ve yoluna devam et.
                pass
# Toplanan tüm şarkı sözlüğü listesini bir Pandas DataFrame'e dönüştür.
# Bu, veriyi temizlemeyi ve zenginleştirmeyi kolaylaştırır.
df_temiz = pd.DataFrame(song_data)
print(f"\n{len(df_temiz)} adet şarkı sözü okundu ve DataFrame oluşturuldu.")


# --- 3. Veri Zenginleştirme (Kritik Adım) ---
# KULLANILAN VERİ SETİ (Kaggle) SADECE ŞARKI SÖZÜ VE KATEGORİ İÇERİR,
# ANCAK 'SANATÇI' BİLGİSİNİ İÇERMEZ.
# Chatbot'un "Bu şarkı kimin?" sorusuna cevap verebilmesi için
# 'sanatçi' bilgisini verimize eklememiz gerekiyor.
# Bu projede bu sorun, manuel olarak hazırlanmış bir 'Şarkı: Sanatçı'
# listesi (user_text) üzerinden çözülmüştür.

# Manuel olarak oluşturulmuş "Şarkı Adı: Sanatçı Adı" eşleştirme metni.
# BU LİSTE, PROJENİN DOĞRU ÇALIŞMASI İÇİN TAM VE EKSİKSİZ OLMALIDIR.
user_text = """
Pop ve Rock
Aşkın Mapushane: Haluk Levent
Bir Sevmek Bin Defa: 3 Hürel
Öpmek İsterdim: Haluk Levent
Dağları Deldim: Haluk Levent
Dibine Kadar: Duman
Birileri Var: Duman
Ben Böyleyim: Athena
Kimdir O: Athena
Aman Aman: Duman
Öpücük: Simge
Vazgeçtim Dünyadan: Şebnem Ferah
Cambaz: Mor ve Ötesi
Ben Seni Arayamam: Cem Adrian
Pencere: Cem Adrian
Bir Kadın Çizeceksin: maNga
Sokak Lambası: Yüksek Sadakat
Dursun Zaman: maNga (ft. Göksel)
Mor Yazma: Umut Kaya
Bu Akşam: Duman
Dünyanın Sonuna Doğmuşum: maNga
Can Kırıkları: Şebnem Ferah
Koca Yaşlı: Adamlar
Bitti Rüya: Pinhani
Oyunbozan: Mor ve Ötesi
Senden Daha Güzel: Duman
İyi de Banane: Model
Her Şeyi Yak: Duman
Kime Ne: Duman
Çakıl Taşları: Şebnem Ferah
Cevapsız Sorular: maNga
Saydım: Ogün Sanlısoy
Yaz Yaz Yaz: Ajda Pekkan
Rüyalarda Buruşmuşum: Adamlar
Ben Öldüm: Pinhani
Yıldızların Altında: Kargo
Seni Kendime Sakladım: Duman
Çatal Yürek: Haluk Levent
Islak Islak: Cem Karaca
Sil Baştan: Şebnem Ferah
Sevda Çiçeği: Mor ve Ötesi
Öyle Dertli: Duman
Yaşamak Var Ya: Athena
Arsız Gönül: Athena
Afili Yalnızlık: Emre Aydın
Köprüaltı: Duman
Kelebekler: Pinhani
Tamirci Çırağı: Cem Karaca
Yollarda Bulurum: Haluk Levent
Bu Aşk Fazla: Şebnem Ferah
Yalnız: Duman
Mavi: Pinhani
Haydi Gel İçelim: Duman
Kafama Göre: Athena
Düşler Sokağı: Feridun Düzağaç
Resimdeki Gözyaşları: Cem Karaca
Elleri Ellerime: Duman
Sende Yap: Pinhani
Vurdum En Dibe: Duman
Mayın Tarlası: maNga
Bir Derdim Var: Mor ve Ötesi
Yolla: Tarkan
Sahici: Tarkan
Efkar Gecesi: Duman
Olmaz Deme: Tarkan
Hadi Çal: Tarkan
Nasıl olacak: Tarkan
Hayatım Kaymış: Can Bonomo
Vanilya: Simge
Çağır: Mabel Matiz
Roket: Mabel Matiz
Çınlama: Mabel Matiz
Arıyorum: Edis
Miş Miş: Simge
İsabelle: Simge
Numaracı: Simge
Ayrı Gitme: Aleyna Tilki
Masum: Zeynep Bastık
Zaman: Zeynep Bastık
İltimas: Gülşen ft. Murat Boz
Dudak: Edis
Tövbeler Olsun: Tarkan
Sor: Tarkan
Hoşuna mı Gidiyor: Ece Seçkin ft. Ozan Doğulu
Arada Sırada: Sinan Akçıl
Yaz Gülü: İrem Derici
Papatya: Teoman
Ayıp Yani: Demet Akalın
Ben Olsaydım: Simge
Bu Nasıl Aşk: Hadise
Dudaklarım Yeminli: Hande Yener
Haydi Gel Benimle Ol: Sezen Aksu
Bitter: Hadise
Yani: Tarkan
Kimseler Bulamasın: Mabel Matiz
Ara: Mem Ararat
Sahi: Mabel Matiz
Tabi Tabi: Sinan Akçıl
Al Aramızdan: Köfn
Feryat: Tarkan
Yap Bi Güzellik: Tarkan
Diva Yorgun: Melike Şahin
Seviyo Muyuz: İrem Derici
Uyanda Gel: Mabel Matiz
Nerdesin Aşkım: Hadise
Mevsimsizim: Mabel Matiz
Başıma Belasın: Hande Yener
E S - Aman Aman: Ece Seçkin
Sarışın: Sezen Aksu
Bangır Bangır: Gülşen
Senin Olsun: İrem Derici
Bugün Adım Leyla: Hande Yener
Nefes: Hande Yener
Aslan Gibi: Kurtuluş Kuş
Kaybet: Hande Yener
Nankör: Hande Yener
Aşkın Olayım: Simge
Sana Nolmuş: Hande Yener
Acele Etme: Hande Yener
Olmuşum Leyla: Hande Yener
Harcandıkça: Hande Yener
Sevdanın Tadı: Hande Yener
Arabesk ve Fantezi
Batsın Bu Dünya: Orhan Gencebay
Allahım Neydi Günahım: Kayahan
Dertler Benim Olsun: Orhan Gencebay
Unutamadım: Müslüm Gürses
Duvardaki Resim: Müslüm Gürses
Unutama Beni: Esengül
Birkaç Damla Yaş: Müslüm Gürses
Zor Bela: Müslüm Gürses
Bir Telefon: Müslüm Gürses
Hani Bekleyecektin: Müslüm Gürses
Vazgeç Gönül: Orhan Gencebay
Son Mektup: Müslüm Gürses
Ağlamam Ondan: Müslüm Gürses
Huzurum Kalmadı: Ferdi Tayfur
Sen Aldırma: Ferdi Tayfur
Beni Kaybettin Artık: Ferdi Tayfur
Akşam Güneşi: Sibel Can
Bana Sor: Ferdi Tayfur
Bir Teselli Ver: Orhan Gencebay
Yalan Oldu: Ferdi Tayfur
Dil Yarası: Orhan Gencebay
Ah Gülüm: Müslüm Gürses
Sensiz Olmaz: Müslüm Gürses
Yalnızım: İbrahim Tatlıses
Kul Hatasız Olmaz: Orhan Gencebay
Şimdi Uzaklardasın: Zeki Müren
Sevda Yüklü Kervanlar: Ferdi Tayfur
Yalnızım Dostlarım: İbrahim Tatlıses
Kırılsın Ellerim: Muazzez Ersoy
Elimde Fotoğrafın: Bergen
Kaderimin Oyunu: Orhan Gencebay
K Y - Yanarım: Kayahan
Ağlamak Yok Yüreğim: Hakan Taşıyan
Canısı: İbrahim Erkal
Dilek Taşı: Gülden Karaböcek
Vazgeçtim: Sezen Aksu
Sabahçı Kahvesi: Ferdi Tayfur
Hatıran Yeter: Ferdi Tayfur
Doktor: Müslüm Gürses
Sen Affetsen Ben Affetmem: Bergen
Yorgun Yıllarım: Cengiz Kurtoğlu
Bir Ayrılık Şarkısı: Müslüm Gürses
Ağlıyorsam Yanıyorum: Cengiz Kurtoğlu
Eğer Ağlıyorsam: Cengiz Kurtoğlu
Nilüfer: Müslüm Gürses
Güz Gülleri: Hakan Taşıyan
Unutabilsem: Emrah
Ahmet K - Söyle: Ahmet Kaya
İstemem Seni: Emrah
Neden Saçların Beyazlamış Arkadaş: Adnan Şenses
Bir Ateşe Attın Beni: İbrahim Tatlıses
Merak Etme Sen: Cengiz Kurtoğlu
Hangimiz Sevmedik: Müslüm Gürses
Gitme Sana Muhtacım: Zeki Müren
Acıların Çocuğu: Emrah
Sevme: Emrah
Hazin Geliyor: Emrah
Çare Gelmez: Ferdi Tayfur
Yıllar Utansın: Müslüm Gürses
Benden Bu Kadar: Müslüm Gürses
Rap ve Alternatif
Deli: Ezhel
Arasan da: UZI
Sözler Şerefsiz Oldu: Norm Ender
Aynen: Heijan ft. Muti
Değilim Bi Aşık: UZI
Cindy: UZI
Le Le: UZI
Kayıp Kelimeler: Sagopa Kajmer
Bu Senin Ellerinde: Sagopa Kajmer
Meftun: Canbay & Wolker
Konum Gizli: Heijan ft. Muti
Neyim Var ki: Ceza ft. Sagopa Kajmer
Hep mi Ben: UZI
Zor: UZI
Pişman Değilim: UZI
Suspus: Ceza
Ben Elimi Sana Verdim: Sagopa Kajmer
Baybay: Ceza
Kimsin Sen: Norm Ender
Mekanın Sahibi: Norm Ender
180km: luffex
Sazımı Duvara Astım: Barış Manço
Caney: UZI
Ela: Reynmen
Kalbim Çukurda: Gazapizm
Unutulacak Dünler: Gazapizm
Paparazzi: Murda
Olur Mu: Gazapizm
Aya: UZ4Y
Karanlık Dünyam: Gazapizm
Kafanı Boşalt: Ezhel
Affetmem: Blok3
Yakalarsan: UZI
Gökyüzü: Sagopa Kajmer
Herkes Gibisin: Cem Adrian
Of Aman: Sagopa Kajmer
Nerdesin: Ezhel
Terapi: Ezhel
Hayalin Yeri Yok: Gazapizm
Çıktık Yine Yollara: Gazapizm
Ararım Yarın: Murda & MERO
Baytar: Murda
Vur Keyfin Dibine: Murda
Yerli Plaka: Ceza
Yarın Ölümü Beklemek Yerine: Gazapizm
Holocaust: Ceza
Heyecanı Yok: Gazapizm
İmdat: Çakal
Umrumda Değil: UZI
Bi Sonraki Hayatımda Gel: Murda & Ezhel
Sevecek Sandım: UZI
Cuma: UZI
Krvn: UZI
Naptığını Bilmesem de: UZI
Makina: UZI
İstisnalar Kaideyi Bozmaz: Sagopa Kajmer
Vasiyet: Sagopa Kajmer
Ateşten Gömlek: Sagopa Kajmer
"""

# Bu metni bir Python sözlüğüne (dictionary) dönüştürme işlemi:
sanatci_map = {} # Boş bir 'sanatçı haritası' sözlüğü oluştur.
for line in user_text.strip().split('\n'): # Metni satır satır oku
    if ':' in line: # Sadece ':' içeren (eşleştirme olan) satırları işle
        try:
            # Satırı ':' karakterinden ikiye böl (Şarkı, Sanatçı)
            sarki, sanatci = line.split(':', 1)

            # Sözlüğe ekle: {'Şarkı Adı': 'Sanatçı Adı'}
            # .strip() ile baş/sondaki boşlukları temizle.
            sanatci_map[sarki.strip()] = sanatci.strip()
        except ValueError:
            # Kategori başlıkları ('Pop ve Rock' gibi) ':' içermez
            # veya formatı bozuk satırlar varsa hata vermeden geç.
            pass

# Şimdi 'df_temiz' DataFrame'indeki 'sarki_adi' sütununu
# bu 'sanatci_map' sözlüğü ile eşleştirerek YENİ bir 'sanatci' sütunu oluştur.
df_enriched = df_temiz.copy() # Orijinal DataFrame'i koru, kopyasını al.
df_enriched['sanatci'] = df_enriched['sarki_adi'].map(sanatci_map)

# Eşleşme bulunamayan (sözlükte olmayan) şarkılar için 'sanatci' sütununa 'NaN'
# yerine 'Bilinmiyor' yaz.
df_enriched['sanatci'].fillna('Bilinmiyor', inplace=True)

# DataFrame'i RAG için daha okunaklı bir sıraya koy.
df_enriched = df_enriched[['sanatci', 'sarki_adi', 'kategori', 'sozler']]
print("Veri seti 'sanatçı' bilgisi ile zenginleştirildi.")


# --- 4. Metinleri LangChain Document Nesnelerine Dönüştürme ---
# Vektör veritabanına eklemeden önce, her şarkı sözünü
# LangChain'in anladığı standart bir 'Document' formatına getirmeliyiz.
documents = []
# DataFrame'deki her bir satır (şarkı) için döngü başlat
for index, row in df_enriched.iterrows():
    # 'metadata', şarkı sözünün "künyesidir".
    # Vektör veritabanı benzer bir söz parçası bulduğunda,
    # bu metadata sayesinde o parçanın HANGİ ŞARKIYA ve KİME ait olduğunu bilecek.
    metadata = {
        "sanatci": str(row.get('sanatci')),
        "sarki_adi": str(row.get('sarki_adi')),
        "kategori": str(row.get('kategori'))
    }

    # 'page_content', asıl metnin (şarkı sözünün) kendisidir.
    # 'metadata' ise o metne ait üst veridir.
    doc = Document(page_content=row['sozler'], metadata=metadata)
    documents.append(doc)

print(f"{len(documents)} adet şarkı LangChain Document nesnesine dönüştürüldü.")

# --- 5. Metinleri Parçalama (Splitting) ---
# LLM'lerin (Gemini) bir 'context window' (bağlam penceresi) sınırı vardır.
# Ayrıca, arama yaparken uzun bir şarkı sözünün tamamı yerine
# sadece soruyla ilgili olan kısmını bulmak daha etkilidir.
# Bu yüzden 'Document'ları daha küçük parçalara (chunks) böleriz.

# DÜZELTME: Bu, doğru sonucu almamızı sağlayan en önemli ayarlardan biriydi.
# Parçaları çok büyütmeden, ama anlamlı olacak şekilde bölüyoruz.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Her parçanın maksimum karakter sayısı.
    chunk_overlap=50 # İki parça arasında 50 karakterlik kesişim bırak.
                     # Bu, cümlenin ortadan ikiye bölünmesi durumunda
                     # anlam bütünlüğünün kaybolmasını engeller.
)

# Tüm 'Document'ları (şarkıları) bu ayarlara göre parçala.
splits = text_splitter.split_documents(documents)
print(f"{len(documents)} şarkı, {len(splits)} adet metin parçasına (chunk) bölündü.")


# --- 6. Embedding Modeli Yükleme ---
# 'Embedding', metin parçalarını (sözleri) bilgisayarın anlayabileceği
# anlamsal bir sayı dizisine (vektör) dönüştürme işlemidir.
# "all-MiniLM-L6-v2", bu iş için optimize edilmiş, hızlı ve etkili
# bir 'sentence-transformer' modelidir.
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"'{model_name}' embedding modeli yüklendi.")


# --- 7. Vektör Veritabanı Oluşturma ve Kaydetme ---
# Bu son adımda, oluşturduğumuz tüm 'splits' (metin parçaları)
# 'embeddings' modeli kullanılarak vektöre dönüştürülür ve
# 'Chroma' veritabanına kaydedilir.

vectorstore = Chroma.from_documents(
    documents=splits,           # Vektöre dönüştürülecek metin parçaları
    embedding=embeddings,       # Bu parçaları vektöre dönüştürecek model
    persist_directory="chroma_db" # Sonuçların kaydedileceği klasör adı
)

print("\n✅✅✅ Başarılı! Vektör veritabanı oluşturuldu ve 'chroma_db' klasörüne kaydedildi.")
print("Artık bu notebook'u kapatabilir ve 'app.py' uygulamasını çalıştırabilirsiniz.")

▶️ HÜCRE 3: Veri seti hazırlanıyor ve vektör veritabanı SIFIRDAN oluşturuluyor...
Veri seti 'Lyrics' klasörü zaten mevcut, indirme adımı atlandı.


Şarkılar okunuyor: 5it [00:00, 429.74it/s]


240 adet şarkı sözü okundu ve DataFrame oluşturuldu.
Veri seti 'sanatçı' bilgisi ile zenginleştirildi.
240 adet şarkı LangChain Document nesnesine dönüştürüldü.
240 şarkı, 1277 adet metin parçasına (chunk) bölündü.



/tmp/ipython-input-257716938.py:368: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_enriched['sanatci'].fillna('Bilinmiyor', inplace=True)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your not

'sentence-transformers/all-MiniLM-L6-v2' embedding modeli yüklendi.

✅✅✅ Başarılı! Vektör veritabanı oluşturuldu ve 'chroma_db' klasörüne kaydedildi.
Artık bu notebook'u kapatabilir ve 'app.py' uygulamasını çalıştırabilirsiniz.


In [2]:
# ==============================================================================
# HÜCRE 4: RAG ZİNCİRİNİN KURULMASI VE TEST EDİLMESİ
# Amaç: Hücre 3'te oluşturduğumuz kalıcı 'chroma_db' veritabanını geri yüklemek
#       ve LangChain Expression Language (LCEL) kullanarak modern, esnek bir
#       RAG (Retrieval-Augmented Generation) zinciri (chain) kurmak.
#       Bu hücre, 'app.py' dosyasında kullanacağımız RAG mantığının
#       Colab üzerinde bir testidir.
# ==============================================================================

print("▶️ HÜCRE 4: Modern zincir ile chatbot kuruluyor...")

# LCEL (LangChain Expression Language) için gerekli temel bileşenler
from langchain_core.prompts import PromptTemplate
    # LLM'e (Gemini) göndereceğimiz talimat şablonunu oluşturmak için.
from langchain_core.runnables import RunnablePassthrough
    # Kullanıcının sorusu gibi, bir girdiyi zincirin sonraki adımlarına
    # HİÇ DEĞİŞTİRMEDEN "olduğu gibi" aktarmak için kullanılır.
from langchain_core.output_parsers import StrOutputParser
    # LLM'in cevabını (genellikle bir 'ChatMessage' nesnesidir)
    # bizim için basit bir 'string' (metin) haline getiren çıktı ayrıştırıcı.


# --- 1. Veritabanını (Vectorstore) Yükleme ---
# (Bu hücreyi çalıştırmadan önce HÜCRE 3'ün başarıyla bitmiş olması gerekir)
# 'chroma_db' klasöründeki kalıcı veritabanını hafızaya yüklüyoruz.

# Embedding modelini tekrar tanımlamalıyız, çünkü Chroma'nın
# veritabanını yüklerken hangi modelle oluşturulduğunu bilmesi gerekir.
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# 'persist_directory' ile veritabanını diskten yükle.
vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embeddings)
print("Kalıcı 'chroma_db' veritabanı başarıyla yüklendi.")


# --- 2. Retriever'ı (Getirici) Yapılandırma ---
# 'Retriever', veritabanında (vectorstore) arama yapma mantığını tanımlar.

# DÜZELTME: Bu, LLM'in kafasının karışmasını önleyen en kritik ayardı.
# 'search_kwargs={"k": 1}' : Kullanıcının sorusuna en çok benzeyen
# SADECE 1 (bir) adet metin parçasını (chunk) veritabanından getir.
#
# NEDEN k=1?
# Eğer k=3 deseydik, LLM'e 3 farklı (belki alakasız) şarkı sözü parçası
# gönderecektik. Bu, modelin kafasını karıştırıp yanlış cevap
# ("hallucination") vermesine veya "Şu şarkı da olabilir, bu da..."
# gibi belirsiz cevaplar vermesine neden oluyordu.
# k=1'e odaklanmak, cevabın doğruluğunu ve netliğini maksimize etti.
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


# --- 3. Prompt Şablonunu (Talimat Metni) Oluşturma ---
# Bu, RAG'ın "Augmented" (Geliştirme/Yönlendirme) kısmıdır.
# LLM'e (Gemini) ne yapması gerektiğini tam olarak söylediğimiz talimat metni.
template = """
Sana verilen Bağlam'ı kullanarak kullanıcının Soru'sunu cevapla. Cevabını SADECE bu bağlamdaki bilgilere dayanarak ver. Eğer bağlamda cevap yoksa, 'Bu konuda bilgim yok.' de.

Bağlam:
{context}

Soru: {question}

Cevap:
"""
# 'context' ve 'question' değişkenleri, zincir çalışırken
# otomatik olarak doldurulacak yer tutuculardır.
# 'SADECE' ve 'Bu konuda bilgim yok' talimatları, modelin
# veritabanında olmayan bilgileri uydurmasını (hallucination) engeller.
prompt = PromptTemplate.from_template(template)


# --- 4. LLM'i (Dil Modeli) Yükleme ---
# DÜZELTME: Hesabının kullanabildiği, çalışan doğru model adını yazıyoruz.
llm = ChatGoogleGenerativeAI(
    model="gemini-pro-latest",  # Kullanılacak Google Gemini modelinin adı.
    temperature=0               # Modelin "yaratıcılık" seviyesi.
                                # '0', modelin en olası ve gerçekçi cevabı
                                # vermesini sağlar (uydurma yapmaz).
                                # Sohbet için değil, RAG için 0 idealdir.
)


# --- 5. Tüm Parçaları Birleştirerek Modern Zinciri Oluşturma (LCEL) ---
# LangChain Expression Language (LCEL), '|' (pipe) operatörünü kullanarak
# adımları birbirine bağladığımız modern bir "boru hattı" sistemidir.
# Okunuşu: "Önce şunu yap, ÇIKTISINI al, bir sonrakine GİRDİ olarak ver."

rag_chain = (
    # ADIM 1: Girdileri Hazırla (Paralel Çalışır)
    # Kullanıcı bir 'soru' (string) ile zinciri çağırır.
    # Bu sözlük, o soruyu iki yere birden gönderir:
    {
     "context": retriever,           # 1. Soru 'retriever'a gider, 'context' (şarkı sözü) bulunur.
     "question": RunnablePassthrough() # 2. Soru 'olduğu gibi' bir sonraki adıma aktarılır.
    }

    # ADIM 2: Prompt'u Doldur
    # '|' (pipe): Adım 1'in çıktısı ('context' ve 'question') alınır
    # ve 'prompt' şablonuna GİRDİ olarak verilir.
    # Çıktı: Doldurulmuş, LLM'e hazır talimat metni.
    | prompt

    # ADIM 3: LLM'i Çalıştır
    # '|' (pipe): Doldurulmuş prompt alınır ve 'llm'e (Gemini) GİRDİ olarak verilir.
    # Çıktı: LLM'in cevabını içeren bir 'ChatMessage' nesnesi.
    | llm

    # ADIM 4: Çıktıyı Temizle
    # '|' (pipe): 'ChatMessage' nesnesi alınır ve 'StrOutputParser'a verilir.
    # Çıktı: Nihai cevap (basit bir string/metin).
    | StrOutputParser()
)

print("✅ Modern chatbot (RAG zinciri) başarıyla kuruldu!")

▶️ HÜCRE 4: Modern zincir ile chatbot kuruluyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Kalıcı 'chroma_db' veritabanı başarıyla yüklendi.
✅ Modern chatbot (RAG zinciri) başarıyla kuruldu!


In [3]:
# ==============================================================================
# HÜCRE 5: ZİNCİRİ TEST ETME (SORU SORMA)
# Amaç: Hücre 4'te kurduğumuz 'rag_chain'in RAG görevini
#       (Metin bulma + Soru yanıtlama) doğru yapıp yapmadığını
#       gerçek bir senaryo ile test etmek.
# ==============================================================================

# --- SORU SORMA KISMI ---

# Test sorumuzu tanımlıyoruz.
# Bu soru, RAG sistemimizin birden fazla yeteneğini test etmek için
# özel olarak seçilmiştir:
# 1. "Aşkın kazanması için ayrı gitme": Bu, veritabanında (ChromaDB)
#    anlamsal arama (semantic search) yapılmasını tetikler.
#    'retriever'ın doğru şarkı sözü parçasını bulması gerekir.
# 2. "...şarkı hangisi, kim söylüyor ve türü nedir?": Bu kısım,
#    LLM'in sadece bulduğu sözü (context) vermekle kalmayıp,
#    o söze bağlı olan 'metadata'yı (Hücre 3'te eklediğimiz
#    sanatci, sarki_adi, kategori) da okuyup, anlayıp,
#    cevabına dahil etmesini test eder.
soru = "Aşkın kazanması için ayrı gitme' sözü geçen şarkı hangisi, kim söylüyor ve türü nedir?"
print(f"\nSoru: {soru}")


# --- ZİNCİRİ ÇALIŞTIRMA ---
# 'rag_chain.invoke(soru)', Hücre 4'te tanımladığımız tüm boru hattını
# (LCEL) baştan sona çalıştırır:
#
# 1. 'soru' alınır -> Retriever'a gider -> 'context' (şarkı sözü parçası) bulunur.
# 2. 'soru' alınır -> RunnablePassthrough'dan geçer -> 'question' olarak kalır.
# 3. 'context' ve 'question' -> PromptTemplate'e gider -> Doldurulmuş talimat metni oluşur.
# 4. Doldurulmuş talimat metni -> LLM'e (Gemini) gider -> Model cevabı üretir (ChatMessage).
# 5. Modelin cevabı -> StrOutputParser'a gider -> Cevap (string) elde edilir.
#
# Tüm bu işlemlerin sonucu 'cevap' değişkenine atanır.
cevap = rag_chain.invoke(soru)


# --- SONUCU GÖSTERME ---
print("\n--- CHATBOT'UN CEVABI ---")
# LLM'in (Gemini) ürettiği ve bizim Prompt ile yönlendirdiğimiz
# ("SADECE bağlamı kullan...") nihai cevabı yazdır.
print(cevap)


Soru: Aşkın kazanması için ayrı gitme' sözü geçen şarkı hangisi, kim söylüyor ve türü nedir?

--- CHATBOT'UN CEVABI ---
Şarkının adı 'Ayrı Gitme', sanatçısı Aleyna Tilki ve türü Pop'tur.


In [4]:
!zip -r chroma_db_GUNCEL.zip chroma_db

  adding: chroma_db/ (stored 0%)
  adding: chroma_db/chroma.sqlite3 (deflated 56%)
  adding: chroma_db/29be6c24-7568-4bf2-8326-0651a474c28b/ (stored 0%)
  adding: chroma_db/29be6c24-7568-4bf2-8326-0651a474c28b/header.bin (deflated 58%)
  adding: chroma_db/29be6c24-7568-4bf2-8326-0651a474c28b/index_metadata.pickle (deflated 48%)
  adding: chroma_db/29be6c24-7568-4bf2-8326-0651a474c28b/data_level0.bin (deflated 12%)
  adding: chroma_db/29be6c24-7568-4bf2-8326-0651a474c28b/link_lists.bin (deflated 86%)
  adding: chroma_db/29be6c24-7568-4bf2-8326-0651a474c28b/length.bin (deflated 97%)
